# Чат-бот-«барахольщик», который по продуктовому запросу он будет рекомендовать товары, по остальным запросам он будет отвечать «болталкой» (без фолбека)

### Задача:

* Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).
* Добавить логику поиска похожих товаров по продуктовому запросу.
* Вся логика должна быть завёрнута в метод get_answer(). Ответ на продуктовый запрос должен иметь вид "product_id title".


In [70]:
import os
import string
import annoy
import codecs
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, KeyedVectors

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_colwidth', 100)

In [71]:
# нужно ли использовать описание товара или нет
USE_DESCRIPTION = True

file_with_prepared_answers = 'data/prepared_answers.txt'
file_with_answers = 'data/Otvety.txt'
file_with_products = 'data/ProductsDataset.csv'

if not os.path.isfile(file_with_answers) and not os.path.isfile(file_with_prepared_answers):
    raise Exception(f'Требуется файл с ответами: {file_with_answers}')
if not os.path.isfile(file_with_products):
    raise Exception(f'Требуется файл с информацией о продуктах: {file_with_products}')

In [72]:
# Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее.
# Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста
if not os.path.isfile(file_with_prepared_answers):
    question = None
    written = False

    # Мы идем по всем записям, берем первую строку как вопрос
    # и после знака --- находим ответ
    with codecs.open(file_with_prepared_answers, "w", "utf-8") as fout:
        with codecs.open(file_with_answers, "r", "utf-8") as fin:
            for line in tqdm(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [73]:
df = pd.read_csv(file_with_products)
df.head(5)

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем на фото.",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58/e3/58e3cf64f567959c58179822.jpg
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но маломерят на 39. Каблук 13 см.\nНебольшой дефект н...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota_kabluka': 'Высокий', 'zhenskaya_obuv_razmer': '40...",http://cache3.youla.io/files/images/360_360/5b/46/5b4636bdf2350229f21b5222.jpg
2,Брюки,"Размер 40-42. Брюки почти новые - не знаю как мерила при покупке, но носить не смогла, т.к. узки...",59534826aaab284cba337e06,9.0,906,"{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюки', 'zhenskie_dzhinsy_razmer': '26'}",http://cache3.youla.io/files/images/360_360/59/53/595346b8132ca52a41057002.jpg
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_aksessuary_tip': 'Головные уборы'}",http://cache3.youla.io/files/images/360_360/57/de/57de54329a64a213958f61d8.jpg
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как новая! \nСмотрите профиль,много интересного!)\nНар...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskaya_odezhda_razmer': '42-44 (S)', 'bluzy_rubashki_ti...",http://cache3.youla.io/files/images/360_360/5a/d4/5ad4d21ed677502efb18ed23.jpg


In [74]:
# преообразуем данные с товарами, чтобы признак title содержал и старое значение title,
# и новое значение, взятое со столбца descrirption (датасет содержит такой столбец с ошибкой в названии)

if USE_DESCRIPTION:
    df = pd.read_csv(file_with_products)
    df = df[['title', 'descrirption', 'product_id']].copy()

    # Создаем копию датафрейма с замененным столбцом title на description
    df_desc = df.copy()
    df_desc['title'] = df_desc['descrirption']

    # Удаляем столбец description из df_desc, так как он нам больше не нужен
    df_desc.drop('descrirption', axis=1, inplace=True)

    # Объединяем два датафрейма
    new_df = pd.concat([df, df_desc], ignore_index=True)

    # Удаляем столбец description из нового датафрейма, так как он нам не нужен в конечном результате
    new_df.drop('descrirption', axis=1, inplace=True)

    # Добавляем столбец label со значением 1 для всех строк
    new_df['label'] = 1

    df = new_df.copy()

else:
    df = pd.read_csv(file_with_products)
    df = df[['title', 'product_id']].copy()
    df['label'] = 1

df


,title,product_id,label
0,Юбка детская ORBY,58e3cfe6132ca50e053f5f82,1
1,Ботильоны,5667531b2b7f8d127d838c34,1
2,Брюки,59534826aaab284cba337e06,1
3,Продам детские шапки,57de544096ad842e26de8027,1
4,Блузка,5ad4d2626c86cb168d212022,1
...,...,...,...
71091,Юбка Белая по.Турция фирма adL,5b5f181c62e1c6616a7f6472,1
71092,Новый с бирками пиджак размер S в стиле Coco Chanel,5bd6c8b29e94ba033d31f8d0,1
71093,"Женская зимняя спортивная куртка фирмы Rossiqnol,ткань мембрана,очень теплая,брала в Карине за 8...",5bd6c8bc074b3e1c056f69b2,1
71094,Женская ветровка размер 44-46. Цвет приглушённый золотой. Подкладка сетка. Не носилась. Причина ...,5bd6c8fb2138bbc55745362c,1


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71096 entries, 0 to 71095
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       69085 non-null  object
 1   product_id  71072 non-null  object
 2   label       71096 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [76]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69070 entries, 0 to 69069
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       69070 non-null  object
 1   product_id  69070 non-null  object
 2   label       69070 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


In [77]:
# Сформируем второй набор данных из подготовленного списка ответов для "болталки" с таким же
# колличеством данных для сбалансированости классов

df_chat = pd.read_csv(
    file_with_prepared_answers,
    usecols=[0, 1],
    sep="\t",
    nrows=df.shape[0],
    header=None,
)

# Переименование столбцов. Вместо столбца `answer` используется `product_id` для синхронизации с продуктовым набором данных
df_chat.columns = ['title', 'product_id']

# Удаление первой строки (индекс 0)
df_chat = df_chat.drop(index=0).reset_index(drop=True)

# Добавляем столбец label со значением 0 для всех строк
df_chat['label'] = 0

df_chat


,title,product_id,label
0,"Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .",меня вобще прикалывает эта тема :).,0
1,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Т...",0
2,Эбу в двенашке называется Итэлма что за эбу? .,"ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он пр...",0
3,академия вампиров. сколько на даный момент частей книги академия вампиров? .,"4. Охотники и Жертвы, Ледяной укус, Поцелуй тьмы, Кровная клятва.",0
4,как защититься от энергетического вампира .,"Защита мыслью. <br>Каждый человек должен в отношении вампиров взять для себя за правило: ""Нам не...",0
...,...,...,...
69064,История возникновения Корсета для Женщин? Как это отражалось на здоровье? Нужны ли Корсеты?? .,"Король женского гардероба на протяжении нескольких веков. Его свергали с пьедестала в небытие, н...",0
69065,"каковы основные признаки понятия ""цивилизация"" .",Понятие цивилизация обычно используется в нескольких значениях. Наиболее общим из них является о...,0
69066,"помогите росставить коэфициенты NaOH+CO2?,NaOh+Cl?,NaOH+CuSO4?,Zn(OH)+Na2O?,Zn(OH)2+HCl? ,Zn(OH)...","<p>1)</p> Если СО2 много, то кислая соль гидрокарбонат натрия: <br>CO2 + NaOH = NaHCO3 <br>А есл...",0
69067,как преобразовать mp4 видео в формат для iPod nano? посоветуйте какую-нибудь быструю программу .,Лучше всего скачать себе Здесь БЕСПЛАТНЫЙ набор программ! Всего 20 Мб <br>Этот набор содержит 20...,0


In [78]:
# Предобработаем текст: удаляем знаки препинания и делаем лемматизацию

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [79]:
df['title_ls']= df['title'].apply(lambda x: preprocess_txt(x))

In [80]:
df_chat['title_ls'] = df_chat['title'].apply(lambda x: preprocess_txt(x))

### Использование TfidfVectorizer

In [81]:
# Объединим данный в один датафрейм
data = pd.concat([df, df_chat]).reset_index(drop=True)

# Преобразование списков слов обратно в строки, чтобы применить TfidfVectorizer,
data['title_txt'] = data['title_ls'].apply(lambda x: ' '.join(x))

# Векторизация текста
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['title_txt'])
y = data['label']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели логистической регрессии
model = LogisticRegression()
model.fit(X_train, y_train)

# Прогнозирование
predictions = model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, predictions)
print(f'Точность модели: {accuracy:.2f}')

# сохраняем модель
with open('data/model_lr_with_tfidf.pkl', 'wb') as output:
    pickle.dump((model, vectorizer), output)

Точность модели: 0.96


### Использование Word2Vec

Сравним `Word2Vec` модель с моделью векторизации `TfidfVectorizer`.

In [82]:
# Подготовка данных для Word2Vec
sentences = [title.split() for title in data['title_txt']]

# Обучение модели Word2Vec
model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def document_vector(word2vec_model, doc):
    """
    Вычисляет вектор документа путем усреднения векторов содержащихся в нем слов.

    Args:
    model (KeyedVectors): Модель Word2Vec для преобразования слов в векторы.
    doc (list): Список слов документа.

    Returns:
    np.array: Вектор документа.
    """
    # удаление слов, которых нет в модели
    doc = [word for word in doc if word in word2vec_model.wv]
    return np.mean(word2vec_model.wv[doc], axis=0) if doc else np.zeros(word2vec_model.vector_size)

# Векторизация документов
doc_vectors = np.array([document_vector(model_w2v, words) for words in sentences])

# Целевая переменная
y = data['label']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, y, test_size=0.2, random_state=42)

# Обучение модели логистической регрессии
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Прогнозирование
predictions = lr_model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, predictions)
print(f'Точность модели: {accuracy:.2f}')


Точность модели: 0.94


/Users/moskalen/Documents/mine/IDE/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Выводы
* при использование векторизации `TfidfVectorizer`, модель логистической регрессии показала лучший результат (accuracy = 0.96), чем при использовании векторизации `Word2Vec` (accuracy = 0.94)

## Создание индекса для поиска товаров/ответов

In [83]:

def create_index(label_value, index_filename):
    """
    Создает и сохраняет Annoy индекс для быстрого поиска ближайших соседей.

    Args:
    condition (int): Значение в столбце 'label', по которому фильтруются данные.
                     Используется для выбора подмножества данных по определенному признаку.
    index_filename (str): Путь к файлу, в который будет сохранен индекс. Это позволяет
                          загружать индекс без необходимости его повторного построения.

    Пример использования:
    >>> create_index(1, 'data/products_index.ann')
    Создаст индекс для товаров с label 1 и сохранит его в 'data/products_index.ann'.
    """
    index_map = {}
    counter = 0

    # Фильтрация данных по метке и подготовка предложений
    filtered_data = data[data['label'] == label_value]
    sentences = [title.split() for title in filtered_data['title_txt'] if len(title) > 2]

    model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
    index = annoy.AnnoyIndex(100, 'angular')

    for _, row in filtered_data.iterrows():
        n_w2v = 0
        index_map[counter] = (row['product_id'], row['title'])
        question = preprocess_txt(row['title'])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)

        if counter == 0:
            print(f'label: {label_value}')
            print(f' - product: {row["product_id"]}')
            print(f' - title: {row["title"]}')
            print(f'question: {question}')

        counter += 1

    index.build(10)
    index.save(index_filename)
    return index_map, model


# Обучение модели и создание индекса для товаров
index_map_products, model_w2v_products = create_index(1, 'data/products_index.ann')

# Обучение модели и создание индекса для ответов
index_map_answers, model_w2v_answers = create_index(0, 'data/answers_index.ann')


label: 1
 - product: 58e3cfe6132ca50e053f5f82
 - title: Юбка детская ORBY
question: ['юбка', 'детский', 'orby']
label: 0
 - product: меня вобще прикалывает эта тема :). 
 - title: Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .
question: ['парень', 'относиться', 'цветной', 'линза', 'девушка', 'зелёный', 'глаз', 'голубой']


## Применение модели и индексов

In [84]:
with open('data/model_lr_with_tfidf.pkl', 'rb') as f:
    model, vectorizer = pickle.load(f)

model_w2v_products = Word2Vec.load('data/w2v_model_1')
model_w2v_answers = Word2Vec.load('data/w2v_model_0')

# Загрузка индексов
index_products = annoy.AnnoyIndex(100, 'angular')
index_products.load('data/products_index.ann')
index_answers = annoy.AnnoyIndex(100, 'angular')
index_answers.load('data/answers_index.ann')

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)


def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls


def find_answer(text, index, index_map, model):
    words = preprocess_txt(text)
    n_w2v = 0
    vector = np.zeros(100)
    for word in words:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]


def get_answer(text):
    predict = model.predict(vectorizer.transform([text]))
    preprocessed_text = preprocess_txt(text)

    if predict == 1:
        product_id, title = find_answer(' '.join(preprocessed_text), index_products, index_map_products, model_w2v_products)
        return product_id + ' ' + title
    else:
        answer, _ = find_answer(' '.join(preprocessed_text), index_answers, index_map_answers, model_w2v_answers)
        return answer


In [85]:
get_answer("Юбка детская ORBY")

'5a5dcdaca09cd57a28447664 Спортивный костюм / для отдыха/ женский р.54-56 Куртка прикрывает бедра. Новый. Велюр. Покупателю подарок.'

In [86]:
get_answer("Где ключи от танка")

'при нажатии на кнопку, создавался файл на хостинге<br><br>Это и есть создание файла на сервере. '